In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

LABELS = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'silence', 'unknown']
ID2NAME = {i: name for i, name in enumerate(LABELS)}
NAME2ID = {name: i for i, name in ID2NAME.items()}


In [135]:
df = pd.read_csv('../output/preds/subm_ensemble_weights_raw_8.csv', index_col='fname')
probs = df.values
fnames = df.index

labels = np.argmax(probs, axis=1)
labels = np.array([ID2NAME[i] for i in labels])
max_probs = np.max(probs, axis=1)

In [146]:
print(len(fnames))
print(len(fnames[max_probs > 0.9]))
print(len(fnames[max_probs <= 0.9]))

158538
157343
1195


In [138]:
fnames[max_probs <= 0.2]

Index(['clip_331554507.wav', 'clip_4163f1c56.wav', 'clip_4a829bd01.wav',
       'clip_4b3f2f2a5.wav', 'clip_5b9466944.wav', 'clip_6984d7385.wav',
       'clip_8c0432a65.wav', 'clip_8c5e9681c.wav', 'clip_9482d2362.wav',
       'clip_9cf04943d.wav', 'clip_a670a03cc.wav', 'clip_b226454a9.wav',
       'clip_d821d2a9b.wav', 'clip_dfb8a2c84.wav', 'clip_ef5fc8148.wav',
       'clip_f8d0d10ab.wav'],
      dtype='object', name='fname')

In [139]:
labels[max_probs <= 0.2]

array(['stop', 'on', 'stop', 'no', 'unknown', 'stop', 'up', 'stop', 'yes',
       'stop', 'on', 'go', 'go', 'unknown', 'stop', 'off'], dtype='<U7')

In [47]:
pseudo_label = pd.DataFrame({'fname': fnames[max_probs > 0.9], 'label': labels[max_probs > 0.9]})
no_label = pd.DataFrame({'fname': fnames[max_probs <= 0.9]})
pseudo_label.to_csv('../input/pseudo_label.csv', index=False)
no_label.to_csv('../input/no_label.csv', index=False)

In [59]:
from utilities import *

pl_fnames = pseudo_label.fname
pl_fnames = np.array([os.path.join(TEST_DIR, f) for f in pl_fnames])

In [60]:
pl_fnames[:10]

array(['C:/data/tf_speech/test/audio/clip_000044442.wav',
       'C:/data/tf_speech/test/audio/clip_0000adecb.wav',
       'C:/data/tf_speech/test/audio/clip_0000d4322.wav',
       'C:/data/tf_speech/test/audio/clip_0001d1559.wav',
       'C:/data/tf_speech/test/audio/clip_0002256ed.wav',
       'C:/data/tf_speech/test/audio/clip_0002a4a1f.wav',
       'C:/data/tf_speech/test/audio/clip_0002d9b83.wav',
       'C:/data/tf_speech/test/audio/clip_000373a5b.wav',
       'C:/data/tf_speech/test/audio/clip_0003e6aee.wav',
       'C:/data/tf_speech/test/audio/clip_00049951d.wav'], dtype='<U47')

In [74]:
from sklearn.model_selection import KFold
import pickle
import gzip

MY_KFOLD_NOISE_FILENAME = 'kfold4_max_new_noise.pklz'
folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + MY_KFOLD_NOISE_FILENAME, 'rb'))

kf = KFold(n_splits=4, shuffle=True, random_state=17)

pl_folds = []
for train_index, test_index in kf.split(pl_fnames):
    pl_folds.append([pl_fnames[train_index], pl_fnames[test_index]])

for f in pl_folds:
    print(len(f[0]), len(f[1]))
    
with gzip.open(TRAIN_MODIFIED_DIR + 'kfold4_max_pseudo_label.pklz', 'wb') as f:
    pickle.dump(pl_folds, f)

86682 28894
86682 28894
86682 28894
86682 28894


In [64]:
pl_fnames[:10]

array(['C:/data/tf_speech/test/audio/clip_000044442.wav',
       'C:/data/tf_speech/test/audio/clip_0000adecb.wav',
       'C:/data/tf_speech/test/audio/clip_0000d4322.wav',
       'C:/data/tf_speech/test/audio/clip_0001d1559.wav',
       'C:/data/tf_speech/test/audio/clip_0002256ed.wav',
       'C:/data/tf_speech/test/audio/clip_0002a4a1f.wav',
       'C:/data/tf_speech/test/audio/clip_0002d9b83.wav',
       'C:/data/tf_speech/test/audio/clip_000373a5b.wav',
       'C:/data/tf_speech/test/audio/clip_0003e6aee.wav',
       'C:/data/tf_speech/test/audio/clip_00049951d.wav'], dtype='<U47')

In [72]:
splits = pl_fnames[0].split('/')
splits

['C:', 'data', 'tf_speech', 'test', 'audio', 'clip_000044442.wav']

In [69]:
pseudo_label.label.value_counts()

unknown    69004
yes         4948
stop        4763
right       4457
off         4454
left        4435
on          4338
no          4069
down        4043
silence     3827
go          3693
up          3545
Name: label, dtype: int64

In [70]:
np.unique(labels, return_counts=True)

(array(['down', 'go', 'left', 'no', 'off', 'on', 'right', 'silence',
        'stop', 'unknown', 'up', 'yes'], dtype='<U7'),
 array([ 5527,  5452,  6088,  6065,  5936,  5904,  5413,  8801,  5544,
        92873,  5384,  5551], dtype=int64))

In [104]:
files_no_pl = pd.read_csv('../input/no_label.csv').fname
labels = np.zeros_like(files_no_pl)
df_no_pl = pd.DataFrame({'label': labels}, index=files_no_pl)

df_pl = pd.read_csv('../input/pseudo_label.csv', index_col='fname')
files_pl = df_pl.index

files_all = glob.glob(os.path.join(TEST_DIR, '*.wav'))
files_all = [f.split('\\')[-1] for f in files_all]

c_pl = 0
c_npl = 0
for fname in files_all:
    if fname in files_pl:
        label = df_pl.loc[fname].label
        c_pl += 1
    else:
        label = df_no_pl.loc[fname].label
        c_npl += 1
        
print(c_pl, c_npl)

115576 42962


In [106]:
files_all = glob.glob(os.path.join(TEST_DIR, '*.wav'))
files_all = [f.split('\\')[-1] for f in files_all]
files_all[:10]

['clip_000044442.wav',
 'clip_0000adecb.wav',
 'clip_0000d4322.wav',
 'clip_0000fb6fe.wav',
 'clip_0001d1559.wav',
 'clip_0002256ed.wav',
 'clip_0002a4a1f.wav',
 'clip_0002d9b83.wav',
 'clip_000373a5b.wav',
 'clip_0003c7122.wav']

In [108]:
pl_folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + 'kfold4_max_pseudo_label.pklz', 'rb'))

In [115]:
for fold in pl_folds:
    for f in fold[0]:
        if f in fold[1]:
            print(f)
        assert f not in fold[1]
    for f in fold[1]:
        if f in fold[0]:
            print(f)
        assert f not in fold[0]